__Домашняя работа к уроку №6__

1. Взять любой набор данных для бинарной классификации

Я возьму датасет про вероятность проблем с сердцем из третьего урока.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
!wget 'https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G' -O train_case2.csv

--2022-04-12 13:36:11--  https://drive.google.com/uc?export=download&id=1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G
Resolving drive.google.com (drive.google.com)... 142.250.152.100, 142.250.152.139, 142.250.152.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.152.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hrpmd0sbsjf5vlnjm96picu8r4ik818m/1649770500000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download [following]
--2022-04-12 13:36:12--  https://doc-00-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hrpmd0sbsjf5vlnjm96picu8r4ik818m/1649770500000/14904333240138417226/*/1Si4EJ_RexI3Q7yZU8eLjgp4ORe_BXr4G?e=download
Resolving doc-00-c0-docs.googleusercontent.com (doc-00-c0-docs.googleusercontent.com)... 142.250.152.132, 2607:f8b0:4001:c56::84
Connecting to doc-00-c0-docs.googleusercontent.com (doc-00-c0

In [3]:
df = pd.read_csv('train_case2.csv', ';')
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


2. Обучить любой классификатор (какой вам нравится)

Я выбрал модель логистической регрессии

Напишем классы для работы с числовыми и категориальными признаками через пайплайн.

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [5]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

Разбиваем выборку на тренировочную и тестовую части и обучаем модель

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), 
                                                    df['cardio'], test_size=0.3, random_state=42)

In [7]:
feature_processing.fit_transform(X_train)

array([[ 0.38817489, -0.5308834 , -0.70816849, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.30846829,  0.32314728, -0.63861364, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.34652706, -0.5308834 , -0.56905879, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67245587,  0.56715605, -0.01261999, ...,  0.        ,
         0.        ,  1.        ],
       [-0.49851342,  0.32314728, -0.29083939, ...,  0.        ,
         0.        ,  0.        ],
       [-1.75890645,  1.54319112, -0.70816849, ...,  0.        ,
         0.        ,  1.        ]])

In [8]:
#Добавим классификатор
classifier = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=42)),
])

# обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_score = classifier.predict_proba(X_test)[:, 1]

Посчитаем precision/recall/f_score и сохраним результат

In [9]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
fscore = 2 * (precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')
result_1 = [fscore[ix], precision[ix], recall[ix]]

Best Threshold=0.41010268477807116, F-Score=0.734, Precision=0.663, Recall=0.824


3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные примеры (класс 1), а только лишь часть.

Посмотрим, сколько в нашем датасете объектов целевого класса.

In [10]:
df['cardio'].value_counts()

0    35021
1    34979
Name: cardio, dtype: int64

Как видем классы объектов распределены примерно поровну. Возьмем первые 5000 объектов из датасета и разметим их как объекты класса "0" в независимости от их реального класса.

In [15]:
df_N = df.iloc[:4999,:]
df_N.loc['cardio'] = 0

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [18]:
df_N.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,0
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,0
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,0
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


Из следующих 10000 объектов возьмем экземпляры только первого класса. 

In [22]:
df_P = df.iloc[5000:14999,:].loc[df['cardio'] == 1]

In [25]:
df_P.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
5002,7082,17590,1,162,104.0,130,80,2,2,0,0,1,1
5004,7084,15128,1,171,86.0,120,80,1,1,0,0,0,1
5005,7087,23525,1,149,56.0,150,90,1,1,0,0,1,1
5006,7089,15857,1,161,77.0,120,70,1,1,0,0,0,1
5007,7090,21761,1,168,77.0,140,80,1,1,0,0,0,1


И составим из получившихся кусков общий датасет для обучения модели по методу RNS.

In [26]:
df_RNS = pd.concat([df_N, df_P], ignore_index = True)

4. Применить random negative sampling для построения классификатора в новых условиях

In [27]:
X_train = df_RNS.drop('cardio', axis=1)
y_train = df_RNS['cardio']

In [28]:
classifier.fit(X_train, y_train)
y_score_RNS = classifier.predict_proba(X_test)[:, 1]

5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик)

In [29]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_RNS)
fscore = 2 * (precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')
result_2 = [fscore[ix], precision[ix], recall[ix]]

Best Threshold=0.44159466490745897, F-Score=0.730, Precision=0.669, Recall=0.803


In [30]:
result_table = pd.DataFrame([result_1, result_2], index = ['LR', 'LR_RNS'], columns = [ 'fscore', 'precision', 'recall'])
result_table

,fscore,precision,recall
LR,0.734388,0.662671,0.823513
LR_RNS,0.729687,0.668562,0.803112


Модель, которая обучалась по методу RNS показала себя практически так же хорошо, как и модель, которая обучалась на реальных данных. Возможно, это связано с тем, что я создал сбалансированную выборку для RNS, где количество объектов обоих классов было примерно равно.